# GRO620 - Activité procédurale 1 - supplément

Reprise et solution aux problèmes Q2.2 et Q5.2.

In [39]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

### Projection (Q2.2)

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$


In [40]:
K = np.array([[620.,   0., 1024.],
              [  0., 620.,  512.],
              [  0.,   0.,    1.]
])



Et les repères ($\{1\}$) correspondant à la caméra): 

![](images_doc/proc1-q2_1-frames.png)

Trouvons les matrices $T_{10}$ (pour passer du repère $\{1\}$ à $\{0\}$) et $T_{01}$ (l'inverse) :

In [41]:
# On troouve la définition du repère {1} à partir du diagramme.
# Ceci correspond à R|T et une dernière rangée (0,0,0,1) pour manipuler
# des coordonnées homogènes.
# Notez qu'on restera en mm pour la suite.

T_10 = np.array([
    [0, 1,  0,  240],
    [1, 0,  0,   80],
    [0, 0, -1,  120],
    [0, 0,  0,    1]
])
print("T_10:\n", T_10)

# Pour l'inverse, on peut transposer la rotation puis déterminer la
# translation depuis le repère {1}, mais on peut tout aussi bien 
# utiliser NumPy pour cet APP: 
T_01 = np.linalg.inv(T_10)
print("T_01:\n", T_01)

T_10:
 [[  0   1   0 240]
 [  1   0   0  80]
 [  0   0  -1 120]
 [  0   0   0   1]]
T_01:
 [[   0.    1.    0.  -80.]
 [   1.    0.    0. -240.]
 [  -0.   -0.   -1.  120.]
 [   0.    0.    0.    1.]]


Soit maintenant le point $p_0 = [0.250, 0.010, 0.000]$ (en m) :

In [42]:
p_0 = np.array([250, 10, 0]) # On convertit en mm

On veut trouver ses coordonnées dans l'image. On peut utiliser l'équation (2.65) du livre. Pour cela, il faut d'abord bâtir Pt (P tilde), qui dépend également de K. Rappel des équations :

$P  = K [R | T]$

$\tilde{P} = \tilde{K} E$

$x_s \sim \tilde{P} \bar{p_w}$

$x_s = (X_s, Y_s, 1, d) $

$d = 1 / z$

Dans notre cas, $E$ correspond à $T_{01}$, car on souhaite passer du repère global à celui de la caméra, $p_w$ est donc $p_0$ et $x_s$ continendra les coordonnées normalisées dans l'image.

$d$ correspond à la disparité, ou $1/z$, où $z$ et la distance du point dans le repère de la caméra (donc en Z, où Z augmente avec la distance).

Nous pouvons maintenant bâtir les matrices :

In [43]:
Kt = np.zeros((4,4)) # On initialise une matrice vide 4x4 pour construire K tilde (eq. 2.64)
Kt[0:3,0:3] = K      # Le premier carré 3x3 correspond à K
Kt[3,3] = 1          # Qu'on complète à la dernière rangée
print("Kt:\n", Kt)

Pt = np.matmul(Kt, T_01) # Équation (2.64)
print("Pt:\n", Pt)

Kt:
 [[6.200e+02 0.000e+00 1.024e+03 0.000e+00]
 [0.000e+00 6.200e+02 5.120e+02 0.000e+00]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00]]
Pt:
 [[ 0.000e+00  6.200e+02 -1.024e+03  7.328e+04]
 [ 6.200e+02  0.000e+00 -5.120e+02 -8.736e+04]
 [ 0.000e+00  0.000e+00 -1.000e+00  1.200e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


On peut maintenant trouver x_s :

In [44]:
p_w = np.array([250,10,0,1]) # On augmente p_0 pour obtenir des coordonnées homogènes

x_s_p = np.matmul(Pt, p_w)    # x_s', avant normalisation
print(x_s_p)

[7.948e+04 6.764e+04 1.200e+02 1.000e+00]


On y est presque ! Il suffit d'obtenir les coordonnées normalisées en divisant Z. Nous avons directement Z dans les coordonnées ici, car elles ont été transposées dans le repère de la caméra :

In [45]:
x_s = x_s_p / x_s_p[2]
print("x_s:", x_s)

x_s: [6.62333333e+02 5.63666667e+02 1.00000000e+00 8.33333333e-03]


Ou, si on arrondit, (662, 564), ce qui correspond à un point dans le quart en bas à gauche de l'image. En regardant les repères, on voit que c'est plausible : le point (0.250, 0.010, 0.000) (en m) dans le repère $\{0\}$ se retrouve un peu plus bas que le centre du repère de la caméra et sur la gauche (vers X-). Il faut cependant comprendre que les coordonnées en pixels commencent tout en haut à gauche de l'image. Un point parfaitement centré avec le repère de la caméra aura les coordonnées (1024, 512). 

### Reprojection (Q5.2)

Il suffit maintenant de faire le chemin inverse.

Premièrement, on peut obtenir l'inverse de la matrice de $\tilde{P}$ :

In [46]:
Pt_inv = np.linalg.inv(Pt)
print("Pt_inv:\n:", Pt_inv)

Pt_inv:
: [[ 0.00000000e+00  1.61290323e-03 -8.25806452e-01  2.40000000e+02]
 [ 1.61290323e-03  0.00000000e+00 -1.65161290e+00  8.00000000e+01]
 [-0.00000000e+00 -0.00000000e+00 -1.00000000e+00  1.20000000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


(Attention: ici, on ne peut pas simplement transposer comme si c'était un repère)

En inversant, on repasse d'un système en pixels à un en mm. Or, on ne peut pas tout de suite multiplier $\tilde{P}^{-1}$ et un vecteur de coordonnées en pixels. On se rappelle que cette matrice ne manipule que les coordonnées normalisées. Il faut donc remultiplier par $z$, ou diviser par $d$ (ce qui correspond à diviser par $1/z$). On peut s'en convaincre avec le point précédent :

In [47]:
x_rp = x_s / x_s[3] # rp pour "reprojection"
print(x_rp) # Devrait corresponde à x_s_p
p_0_rp = np.matmul(Pt_inv, x_rp)
print(p_0_rp)

[7.948e+04 6.764e+04 1.200e+02 1.000e+00]
[250.  10.   0.   1.]


On retrouve bien le point original (en mm) en coordonnées homogènes.

Maintenant, prenons le point $x_s = [120, 200]$, un point dans l'image perçu par la caméra décrite plus haut. Il faut d'abord trouver ses coordonnées homogènes **dans le repère de la caméra**. Dans l'énoncé, on suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Le point a donc $z=0$ dans le repère $\{0\}$, mais $z=120$ dans le repère $\{1\}$. Or, c'est encore la version normalisée dont nous avons besoin. Il faut donc plutôt utiliser $d$:

In [48]:
d = 1.0 / 120  # 1 / z, et z est la distance entre la caméra et le convoyeur, donc 120 mm
x52_s = np.array([120, 200, 1, d])
x52_c = x52_s / d
print(x52_c)
# Et on multiplie :
p52_0 = np.matmul(Pt_inv, x52_c)
print(p52_0)

[1.44e+04 2.40e+04 1.20e+02 1.00e+00]
[179.61290323 -94.96774194   0.           1.        ]


On obtient donc un point franchement vers Y- (ce qui correspond à la gauche de l'image) et à mi-chemin entre les deux repères en X, ce qui a du sens compte tenu des coordonnées de l'image.